In [1]:
import os

import numpy as np
from sklearn.linear_model import LogisticRegression

from utils import load_hidden_representations_from_hdf5, read_templates_from_file

----

In [2]:
# params
log_dir = "/logfiles"
model = "bigscience-T0" # bigscience-T0_B or bigscience-T0
module = "decoder"
task = "rte"
# task = "cb"
# task = "wic"

## Prompts

In [3]:
df = read_templates_from_file(f"/t0-analysis/prompts/{task}/fixed_prompt.csv")
# df = read_templates_from_file(f"/t0-analysis/prompts/{task}/fixed_target_yes_no.csv")
display(df)

,name,template,category,includes_targets,targets,target_ids,shuffle
0,gpt_3_yes_no_with_targets,{premise} Question: {hypothesis} Yes or No?,instructive,True,"▁Yes, ▁No","0, 1",False
1,gpt_3_true_false_with_targets,{premise} Question: {hypothesis} True or False?,instructive,True,"▁True, ▁Fal","0, 1",False
2,gpt_3_cat_dog_with_targets,{premise} Question: {hypothesis} Cat or Dog?,instructive,True,"▁Cat, ▁Dog","0, 1",False
3,gpt_3_yes_no_without_targets,{premise} Question: {hypothesis}?,instructive,False,"▁Yes, ▁No","0, 1",False


## Train linear classifier

In [4]:
def unison_shuffled_copies(a, b):
    # from: https://stackoverflow.com/questions/4601373/better-way-to-shuffle-two-numpy-arrays-in-unison
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

In [5]:
if task == 'rte':
    use_pattern = [
        "gpt_3_yes_no_with_targets",
        "gpt_3_true_false_with_targets",
        # "gpt_3_cat_dog_with_targets",
        # "gpt_3_yes_no_without_targets",
    ]
elif task == 'cb':
    use_pattern = [
        "null_pattern",
        "null_pattern_reversed",
        "gpt_3_true_false_neither",
        "gpt_3_yes_no_maybe",
        "mnli_crowdsource",
        "always_sometimes_never",
        "based_on_previous_passage",
        "infer",
        "claim",
        "consider",
        "follow",
        "imply",
        "guaranteed",
        "guaranteed_possible",
        "justified",
        "must_be_true",
        "should_assume",
        "take_the_following",
    ]
elif task == 'wic':
    use_pattern = [
        "gpt_3",
        "gpt_3_yes_no",
        "affirmation",
        "grammar_homework",
        "polysemous",
        "question_context",
        "question_meaning",
        "question_meaning_yes_no",
        "same_sense",
        "similar_sense",
        "similar_sense_yes_no",
    ]

In [6]:
for t in range(2):
    # for layer in range(0, 1):
    for layer in range(0, 25):
    # for layer in range(24, 25):
        
        print(f"token: {t}\tlayer: {layer}")
        file_names, prompt_names = [], []

        for _, row in df.iterrows():
            if row['name'] in use_pattern:
                file_names.append(f"{task}/{model}/{module}/{row['name']}/hidden_represenations_t{t}_layer{layer}_avg.hdf5",)
                prompt_names.append(row['name'])

        # load hidden representations from hdf5 file
        representations = None
        classes = []
        n_sequences = 0

        for idx, file_name in enumerate(file_names):
            hidden_representations = load_hidden_representations_from_hdf5(os.path.join(log_dir, file_name))
            # print(hidden_representations.shape)
            n_sequences = hidden_representations.shape[0]

            if representations is None:
                representations = hidden_representations
            else:
                representations = np.concatenate((representations, hidden_representations), axis=0)

            classes += n_sequences * [idx] # assign representations to classes
        
        classes = np.asarray(classes)

        # shuffle representations and classes
        X, y = unison_shuffled_copies(representations, classes)
        print(X.shape, y.shape)

        # train linear classifier
        # multi_class='multinomial' uses a CE loss
        print('classifying between:', prompt_names)
        clf = LogisticRegression(random_state=0, max_iter=2000, multi_class='multinomial').fit(X, y)
        
        print(f'token={t}; layer={layer}; accuracy on training data: ', clf.score(X, y))
        print('\n')



token: 0	layer: 0


Reading embeddings: 100%|██████████| 277/277 [00:00<00:00, 5540.51it/s]


(554, 4096) (554,)
classifying between: ['gpt_3_yes_no_with_targets', 'gpt_3_true_false_with_targets']
token=0; layer=0; accuracy on training data:  0.5


token: 0	layer: 1


Reading embeddings: 100%|██████████| 277/277 [00:00<00:00, 4365.06it/s]


(554, 4096) (554,)
classifying between: ['gpt_3_yes_no_with_targets', 'gpt_3_true_false_with_targets']
token=0; layer=1; accuracy on training data:  1.0


token: 0	layer: 2


...

(554, 4096) (554,)
classifying between: ['gpt_3_yes_no_with_targets', 'gpt_3_true_false_with_targets']


token=1; layer=23; accuracy on training data:  1.0


token: 1	layer: 24


Reading embeddings: 100%|██████████| 277/277 [00:00<00:00, 5691.75it/s]

(554, 4096) (554,)
classifying between: ['gpt_3_yes_no_with_targets', 'gpt_3_true_false_with_targets']


token=1; layer=24; accuracy on training data:  1.0


